# FGSM Sweep 1

TJ Kim <br/>
1/16/21

#### Objective: 
Run FGSM attack on different number of head layers in federated learning and observe performance,


In [1]:
cd '/home/ubuntu/FedAtk/' 

/home/ubuntu/FedAtk


### Load Relevant Libraries and Modules

Load the relevant libraries for the federated learning code.

In [2]:
import time
import yaml
        
from federated_training.femnist_dataloader import Dataloader
from federated_training.cnn_head import CNN_Head
from federated_training.cnn_neck import CNN_Neck
from federated_training.cnn_server import Server
from federated_training.cnn_client import Client
from federated_training.data_manager import DataManager
from federated_training.utils import cuda, where

from federated_training.utilities import freeze_layers
import numpy as np
import torch
import matplotlib.pyplot as plt
import random
import csv
import os
import pickle
from torch.autograd import Variable

import multiprocessing as mp

import queue

# Extra not from py file
from collections import OrderedDict 
import itertools

Load the relevant libraries for example FGSM.

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

# Import Custom Made Victim
from transfer_attacks.victim_nn import *

### Transfer Attack Sweeping Different Configs

In [4]:
# Load Config File and Slie Indices
with open(r'configs/config.yaml') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
        
file_indices = [i for i in range(config['num_sets'])]
#random.shuffle(file_indices)
client_slice = len(file_indices)//config['num_clients']

# File names of FL trained setting
filenames = ["exp2_neck2_head1", "exp2_neck2_head2",
             "exp2_neck2_head3", "exp2_neck2_head4"]

# Matrix to Record Performance
orig_acc_transfers = np.zeros((1,config['num_clients']))
orig_similarities = np.zeros((1,config['num_clients']))
orig_target_hit = np.zeros((1,config['num_clients']))
adv_acc_transfers = np.zeros((1,config['num_clients']))
adv_similarities = np.zeros((1,config['num_clients']))
adv_target_hit = np.zeros((1,config['num_clients']))

# Attack Params
batch_size = 1000
eps = 0.5
alpha = 0.5
iteration = 30
target = 10

In [6]:
file = 'exp2_neck2_head3'

for source in range(1):
    
    # Bring in the data loader for this client
    loader = Dataloader(file_indices,[source*(client_slice),min((source+1)*(client_slice),35)])  
    loader.load_training_dataset()
    loader.load_testing_dataset()

    victim_source = load_victim(source,loader,file)

    # Generate adversarial Perturbations
    victim_source.i_fgsm(batch_size = batch_size, target= target, eps=eps, alpha=alpha, 
               iteration=iteration, x_val_min=-1, x_val_max=1, print_info=False)

    # Record relevant tensors
    x_orig = victim_source.x_orig
    y_orig = victim_source.output_orig
    y_true = victim_source.y_orig
    x_adv = victim_source.x_adv
    y_adv = victim_source.output_adv

    print("======== Source", source, "========")

    for dest in range(config['num_clients']):

        print("    ==== Dest", dest, "====")

        victim_dest = load_victim(dest,loader,file)

        # Compute Stats and record
        victim_dest.forward_transfer(x_orig,x_adv,y_orig,y_adv,y_true, target, print_info=True)

        orig_acc_transfers[source,dest] = victim_dest.orig_test_acc
        orig_similarities[source,dest] = victim_dest.orig_output_sim
        orig_target_hit[source,dest] = victim_dest.orig_target_achieve

        adv_acc_transfers[source,dest] = victim_dest.adv_test_acc
        adv_similarities[source,dest] = victim_dest.adv_output_sim
        adv_target_hit[source,dest] = victim_dest.adv_target_achieve

Loading  all_data_12_niid_0_keep_0_train_9.json
Loading  all_data_20_niid_0_keep_0_train_9.json
Loading  all_data_11_niid_0_keep_0_train_9.json
Loading  all_data_18_niid_0_keep_0_train_9.json


/home/ubuntu/anaconda3/lib/python3.8/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


======== Source 0 ========
    ==== Dest 0 ====
---- Attack Transfer: ----

         Orig Test Acc: 0.9130000472068787
          Adv Test Acc: 0.08400000631809235
Orig Output Similarity: 0.984000027179718
 Adv Output Similarity: 0.12200000882148743
       Orig Target Hit: 0.006000000052154064
        Adv Target Hit: 0.03200000151991844
    ==== Dest 1 ====
---- Attack Transfer: ----

         Orig Test Acc: 0.8180000185966492
          Adv Test Acc: 0.09100000560283661
Orig Output Similarity: 0.8540000319480896
 Adv Output Similarity: 0.07900000363588333
       Orig Target Hit: 0.004000000189989805
        Adv Target Hit: 0.018000001087784767
    ==== Dest 2 ====
---- Attack Transfer: ----

         Orig Test Acc: 0.7750000357627869
          Adv Test Acc: 0.09400000423192978
Orig Output Similarity: 0.8110000491142273
 Adv Output Similarity: 0.09200000762939453
       Orig Target Hit: 0.006000000052154064
        Adv Target Hit: 0.02500000037252903
    ==== Dest 3 ====
---- Attack Tran

In [ ]:
print("orig_acc_transfers\n",np.round(orig_acc_transfers,3)[0])
print("orig_similarities\n",np.round(orig_similarities,3))
print("orig_target_hit\n",np.round(orig_target_hit,3))
print("adv_acc_transfers\n",np.round(adv_acc_transfers,3))
print("adv_similarities\n",np.round(adv_similarities,3))
print("adv_target_hit\n",np.round(adv_target_hit,3))